In [ ]:
# https://blogs.sap.com/2020/06/09/connecting-python-with-sap-step-by-step-guide/
# https://github.com/SAP/PyRFC
# https://kminito.tistory.com/77

**SAP BW 자료확보**

+ 이슈사항 : Power BI에서 1.0 구성을 지원하지 않음  [Link](https://learn.microsoft.com/en-us/power-query/connectors/sap-bw/application-setup-and-connect)

+ 기술명칭 : RFC(Remote Function Call)

+ 대응방향 : python을 이용한 별도 DB upoad
  + 참조1 : 깃허브 공식문서 [Link](https://sap.github.io/PyRFC/index.html)
  + 참조2 : [Link](https://blogs.sap.com/2020/06/09/connecting-python-with-sap-step-by-step-guide/)
  + 참조3 : 한글로 된 내용 [Link](https://kminito.tistory.com/77)
  + 참조4 : SAP Function [Link](https://jitterbit.jjackson.design/plausible-documentation/guide-to-using-rfc-read-table-to-query-sap-tables/)
  (https://wiki.scn.sap.com/wiki/display/BI/SAP+Business+Warehouse+Home)
  + 참조5 : Power BI와 같이 자료를 수집하기! [Link](https://learn.microsoft.com/ko-kr/power-query/connectors/sap-bw/sap-bw-troubleshooting)
  + 참조6 : parameter 확인(Metadata) [Link](https://www.sapdatasheet.org/abap/prog/saplrsob.html)

In [1]:
from pyrfc import Connection
import pandas as pd

In [3]:
conn = Connection(ashost='***.**.*.*', sysnr='00', client='001', user='__USER_ID__', passwd='__PASSWORD__', lang = 'KO')

LogonError: RFC_LOGON_FAILURE (rc=2): key=RFC_LOGON_FAILURE, message=00024error during logon [MSG: class=, type=, number=, v1-4:=;;;]

In [3]:
# help(conn.call)

In [4]:
# conn.get_connection_attributes()

In [ ]:
# 연결 확인
result = conn.call('STFC_CONNECTION', REQUTEXT=u'Hello SAP!')
print (result)

In [6]:
# 시스템 정보 확인
result = conn.call('RFC_SYSTEM_INFO')
print (result)

{'CURRENT_RESOURCES': 75, 'MAXIMAL_RESOURCES': 95, 'RECOMMENDED_DELAY': 0, 'RFCSI_EXPORT': {'RFCPROTO': '011', 'RFCCHARTYP': '4103', 'RFCINTTYP': 'LIT', 'RFCFLOTYP': 'IE3', 'RFCDEST': 'maeil-sapbi1_BIP_00', 'RFCHOST': 'maeil-sa', 'RFCSYSID': 'BIP', 'RFCDATABS': 'BIP', 'RFCDBHOST': 'maeil-sapbi1', 'RFCDBSYS': 'DB6', 'RFCSAPRL': '700', 'RFCMACH': '  390', 'RFCOPSYS': 'Linux', 'RFCTZONE': ' 32400', 'RFCDAYST': '', 'RFCIPADDR': '172.16.10.16', 'RFCKERNRL': '722', 'RFCHOST2': 'maeil-sapbi1', 'RFCSI_RESV': '', 'RFCIPV6ADDR': '172.16.10.16'}}


In [5]:
# 소프트웨어 버전 확인?
result = conn.call('OCS_GET_INSTALLED_COMPS')
print (result)

{'ET_COMPLAYER': [{'MODEL_ID': '0000000700', 'COMP_TYPE': 'S', 'LAYER_LVL': '01', 'LAYER_DESC': 'NetWeaver Basis Components'}, {'MODEL_ID': '0000000700', 'COMP_TYPE': 'X', 'LAYER_LVL': '02', 'LAYER_DESC': 'NetWeaver PlugIns'}, {'MODEL_ID': '0000000700', 'COMP_TYPE': 'W', 'LAYER_LVL': '03', 'LAYER_DESC': 'NetWeaver Add-Ons'}, {'MODEL_ID': '0000000700', 'COMP_TYPE': 'V', 'LAYER_LVL': '04', 'LAYER_DESC': 'Application Platform Components'}, {'MODEL_ID': '0000000700', 'COMP_TYPE': 'R', 'LAYER_LVL': '05', 'LAYER_DESC': 'Application Components'}, {'MODEL_ID': '0000000700', 'COMP_TYPE': 'P', 'LAYER_LVL': '06', 'LAYER_DESC': 'Application PlugIns'}, {'MODEL_ID': '0000000700', 'COMP_TYPE': 'O', 'LAYER_LVL': '07', 'LAYER_DESC': 'Common Objects f. Application Extensions'}, {'MODEL_ID': '0000000700', 'COMP_TYPE': 'N', 'LAYER_LVL': '08', 'LAYER_DESC': 'Application Extensions'}, {'MODEL_ID': '0000000700', 'COMP_TYPE': 'I', 'LAYER_LVL': '09', 'LAYER_DESC': 'Industry Solutions - Layer 1'}, {'MODEL_ID': 

In [11]:
# 기본 메타데이터 가져오기
# result = conn.call('BAPI_MDPROVIDER_GET_CUBES')
# print (result)

### 1. MDPROVIDER

In [10]:
# Delivers a List of CUBES 
result = conn.call("BAPI_MDPROVIDER_GET_CUBES", CUBE_NAM='ZSDOPLM07/ZPB_SD_Q002')
result['CUBES'][0]

{'CAT_NAM': 'ZSDOPLM07',
 'CUBE_NAM': 'ZSDOPLM07/ZPB_SD_Q002',
 'CUBE_TYPE': 'QUERY CUBE',
 'CUBE_UID': '48KL1L4KHQ5AROY1J7GYE579S',
 'CREATED_ON': '20221110071355',
 'LST_SCHEMA_UPD': '20221110071351',
 'SCHEMA_UPD_BY': 'POWERBI',
 'LST_DATA_UPD': '20221228030335',
 'DATA_UPD_BY': '',
 'DSCRPTN': '[Business View] 미출현황'}

In [32]:
cube_list = [
             'ZLEST01/ZPB_LE_Q001',
             'ZPPACTM02/ZPB_PP_Q001',
             'ZSDORM06/ZPB_SD_Q001',
             'ZSDOPLM01/ZPB_SD_Q011',
             'ZSDOPLM07/ZPB_SD_Q002',
#              'ZSDSLM10/ZPB_SD_Q004',
#              'ZSDOPLM01/ZPB_SD_Q014',
#              'ZHYPAM12/ZPB_CO_Q001'
            ]
for nm in cube_list:
    result = conn.call("BAPI_MDPROVIDER_GET_CUBES", CUBE_NAM=nm)
    print(result['CUBES'][0]['DSCRPTN'])

[Business View] 재고현황
[Business View] 생산현황
[Business View] 일별 수주현황
[Business View] 수주현황(계획대비실적)
[Business View] 미출현황
[Business View] 일별 매출현황
[Business View] 판매현황(계획대비실적)
[Business View] 손익현황


In [30]:
# Delivers a list of DIMENSIONS  
result = conn.call('BAPI_MDPROVIDER_GET_DIMENSIONS', CUBE_NAM='ZSDORM06/ZPB_SD_Q001')

In [31]:
pd.DataFrame(result['DIMENSIONS']).tail(10)

,CAT_NAM,CUBE_NAM,DIM_NAM,DIM_UNAM,DIM_UID,DIM_CAP,DIM_ORDINAL,DIM_TYPE,DIM_CARDINALITY,DFLT_HRY,DSCRPTN
14,ZSDORM06,ZSDORM06/ZPB_SD_Q001,ZMATERIAL,[ZMATERIAL],8YSSVLCFL80YR0211RLNFCDG0,자재,0014,3,232756,[ZMATERIAL ZMATHIER],자재
15,ZSDORM06,ZSDORM06/ZPB_SD_Q001,ZMATERIAL__0PROD_HIER,[ZMATERIAL__0PROD_HIER],8FM9JLP38SCXGGEZ4U7XUFL00,제품계층구조(N),0015,3,001177,[ZMATERIAL__0PROD_HIER],제품계층구조(N)
16,ZSDORM06,ZSDORM06/ZPB_SD_Q001,ZMATERIAL__ZEIS_L1,[ZMATERIAL__ZEIS_L1],89833LTB4N4X0XVAHV3DBGNIO,EIS 표준계층구조 1레벨,0016,3,000064,[ZMATERIAL__ZEIS_L1],EIS 표준계층구조 1레벨
17,ZSDORM06,ZSDORM06/ZPB_SD_Q001,ZMATERIAL__ZEIS_L2,[ZMATERIAL__ZEIS_L2],873CY9UPR9E8VROQA7DUH50CW,EIS 표준계층구조 2레벨,0017,3,000293,[ZMATERIAL__ZEIS_L2],EIS 표준계층구조 2레벨
18,ZSDORM06,ZSDORM06/ZPB_SD_Q001,ZMATERIAL__ZEIS_L3,[ZMATERIAL__ZEIS_L3],84YMSXW4DVNKQLI62JOBMTD74,EIS 표준계층구조 3레벨,0018,3,002180,[ZMATERIAL__ZEIS_L3],EIS 표준계층구조 3레벨
19,ZSDORM06,ZSDORM06/ZPB_SD_Q001,ZPLANT,[ZPLANT],9329699MBZIB1CF5H30P3ZNRK,플랜트,0019,3,000099,[ZPLANT],플랜트
20,ZSDORM06,ZSDORM06/ZPB_SD_Q001,0CALDAY,[0CALDAY],8BCT8XRWI0VL641UPISW5SAOG,달력일,0020,1,011765,[0CALDAY],달력일
21,ZSDORM06,ZSDORM06/ZPB_SD_Q001,0CALMONTH,[0CALMONTH],8O564XJGQBBM1557ZH217Q5N4,달력연도/월,0021,1,000263,[0CALMONTH],달력연도/월
22,ZSDORM06,ZSDORM06/ZPB_SD_Q001,0CALWEEK,[0CALWEEK],9FUM2916K9YBWDIIR19U5XIQ8,달력연도/주,0022,1,001117,[0CALWEEK],달력연도/주
23,ZSDORM06,ZSDORM06/ZPB_SD_Q001,Measures,[Measures],,주요 지표,0024,2,000004,[Measures],주요 지표


In [32]:
# Delivers a list of DIMENSIONS  
result = conn.call('BAPI_MDPROVIDER_GET_HIERARCHYS', CUBE_NAM='ZSDORM06/ZPB_SD_Q001', DIM_UNAM = '[ZMATERIAL__ZEIS_L1]')

In [33]:
pd.DataFrame(result['HIERARCHIES'])

,CAT_NAM,CUBE_NAM,DIM_UNAM,HRY_NAM,HRY_UNAM,HRY_UID,HRY_CAP,DIM_TYPE,HRY_CARDINALITY,DFLT_MEM,ALL_MEM,DSCRPTN
0,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[ZMATERIAL__ZEIS_L1],,[ZMATERIAL__ZEIS_L1],,EIS 표준계층구조 1레벨,3,000064,[ZMATERIAL__ZEIS_L1].[All],[ZMATERIAL__ZEIS_L1].[All],EIS 표준계층구조 1레벨


In [36]:
result = conn.call('BAPI_MDPROVIDER_GET_MEMBERS', CUBE_NAM='ZSDORM06/ZPB_SD_Q001', DIM_UNAM = '[ZMATERIAL__ZEIS_L1]')

In [46]:
pd.DataFrame(result['MEMBERS']).head(4)

,LVL_NUMBER,MEM_NAM,MEM_UNAM,MEM_TYPE,MEM_UID,MEM_CAP,MEM_ORDINAL,CHILDREN,PARENT_LVL,PARENT_UNAM,DSCRPTN
0,00,All,[ZMATERIAL__ZEIS_L1].[All],02,,모두 EIS 표준계층구조 1레벨,000000,000064,01,,모두 EIS 표준계층구조 1레벨
1,01,00000000000000000000000000010101,[ZMATERIAL__ZEIS_L1].[000000000000000000000000...,01,,유음료 제품군,000001,000000,00,[ZMATERIAL__ZEIS_L1].[All],유음료 제품군
2,01,00000000000000000000000000010201,[ZMATERIAL__ZEIS_L1].[000000000000000000000000...,01,,상하목장 제품군,000002,000000,00,[ZMATERIAL__ZEIS_L1].[All],상하목장 제품군
3,01,00000000000000000000000000010301,[ZMATERIAL__ZEIS_L1].[000000000000000000000000...,01,,Plant& 제품군,000003,000000,00,[ZMATERIAL__ZEIS_L1].[All],Plant& 제품군


In [16]:
result = conn.call('BAPI_MDPROVIDER_GET_MEASURES', CUBE_NAM='ZSDORM06/ZPB_SD_Q001')

In [21]:
pd.DataFrame(result['MEASURES'])

,CAT_NAM,CUBE_NAM,MES_NAM,MES_UNAM,MES_CAP,MES_UID,MES_AGGREGATOR,DATA_TYPE,NUM_PREC,NUM_SCALE,MES_UNITS,DSCRPTN
0,ZSDORM06,ZSDORM06/ZPB_SD_Q001,9M8SI8WYOF6CBW27E0EEOWG7K,[Measures].[9M8SI8WYOF6CBW27E0EEOWG7K],수주수량,,001,FLTP,0,0,,수주수량
1,ZSDORM06,ZSDORM06/ZPB_SD_Q001,9ODINKVK1SX0H28RLO3XJ83DC,[Measures].[9ODINKVK1SX0H28RLO3XJ83DC],환산수주수량(N),,000,FLTP,0,0,,환산수주수량(N)
2,ZSDORM06,ZSDORM06/ZPB_SD_Q001,9QI8SWU5F6NOM8FBTBTGDJQJ4,[Measures].[9QI8SWU5F6NOM8FBTBTGDJQJ4],수주환산용량(N),,001,FLTP,0,3,,수주환산용량(N)
3,ZSDORM06,ZSDORM06/ZPB_SD_Q001,9SMYY8SQSKECRELW0ZIZ7VDOW,[Measures].[9SMYY8SQSKECRELW0ZIZ7VDOW],수주금액,,001,FLTP,0,0,,수주금액


In [8]:
result = conn.call('BAPI_MDPROVIDER_GET_HIERARCHYS', CUBE_NAM='ZSDORM06/ZPB_SD_Q001')

In [10]:
pd.DataFrame(result['HIERARCHIES']).tail(5)

,CAT_NAM,CUBE_NAM,DIM_UNAM,HRY_NAM,HRY_UNAM,HRY_UID,HRY_CAP,DIM_TYPE,HRY_CARDINALITY,DFLT_MEM,ALL_MEM,DSCRPTN
35,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[ZPLANT],,[ZPLANT],,플랜트,3,000099,[ZPLANT].[All],[ZPLANT].[All],플랜트
36,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[0CALDAY],,[0CALDAY],,달력일,1,011765,[0CALDAY].[All],[0CALDAY].[All],달력일
37,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[0CALMONTH],,[0CALMONTH],,달력연도/월,1,000263,[0CALMONTH].[All],[0CALMONTH].[All],달력연도/월
38,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[0CALWEEK],,[0CALWEEK],,달력연도/주,1,001117,[0CALWEEK].[All],[0CALWEEK].[All],달력연도/주
39,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[Measures],,[Measures],,주요 지표,2,000004,[Measures].[9M8SI8WYOF6CBW27E0EEOWG7K],,


In [24]:
result = conn.call('BAPI_MDPROVIDER_GET_STREAMINFO', CUBE_NAM='ZSDORM06/ZPB_SD_Q001')

In [27]:
# pd.DataFrame(result['VARIABLES'])

,CAT_NAM,CUBE_NAM,VAR_NAM,VAR_CAP,VAR_UID,VAR_ORDINAL,VAR_TYPE,DATA_TYPE,CHR_MAX_LEN,VAR_PROC_TYPE,VAR_SELC_TYPE,VAR_ENTRY_TYPE,REF_DIM,REF_HRY,DFLT_LOW,DFLT_HIGH,DFLT_LOW_CAP,DFLT_HIGH_CAP,DSCRPTN
0,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[ZUISAGRP],"영업그룹(복수,선택)",3S4GGCUTTKHYIRCW2GQH9W46I,000001,1,CHAR,143,1,3,0,[ZCUSTOMER__0SALES_GRP],[ZCUSTOMER__0SALES_GRP],,,,,"영업그룹(복수,선택)"
1,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[ZDAY_1],기간일자,47NJGD065TDYVH1DBF5X4EHW2,000002,1,CHAR,143,1,2,1,[0CALDAY],[0CALDAY],,,,,기간일자
2,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[!V000003],EIS 제품계층구조 1레벨,4ASYKSCVW0UTD80EVIAIMKFM8,000003,1,CHAR,143,1,1,0,[ZMATERIAL__ZEIS_L1],[ZMATERIAL__ZEIS_L1],,,,,EIS 제품계층구조 1레벨
3,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[!V000004],EIS 제품계층구조 2레벨,4BBIT97DVP0TP9VKEQSYYYL8W,000004,1,CHAR,143,1,1,0,[ZMATERIAL__ZEIS_L2],[ZMATERIAL__ZEIS_L2],,,,,EIS 제품계층구조 2레벨
4,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[!V000005],EIS 제품계층구조 3레벨,4BBITCELS42GGNZAU9S35S1ZK,000005,1,CHAR,143,1,1,0,[ZMATERIAL__ZEIS_L3],[ZMATERIAL__ZEIS_L3],,,,,EIS 제품계층구조 3레벨


In [6]:
# result = conn.call('BAPI_MDPROVIDER_GET_MEMBERS', CUBE_NAM='ZSDORM06/ZPB_SD_Q001')

In [47]:
result = conn.call('BAPI_MDPROVIDER_GET_PROPERTIES', CUBE_NAM='ZSDORM06/ZPB_SD_Q001', DIM_UNAM = '[ZPLANT]')

In [51]:
pd.DataFrame(result['PROPERTIES']).tail(3)

,CAT_NAM,CUBE_NAM,DIM_UNAM,HRY_UNAM,LVL_UNAM,MEM_UNAM,PRPTY_TYPE,PRPTY_NAM,PRPTY_CAP,DATA_TYPE,CHR_MAX_LEN,CHR_OCT_LEN,NUM_PREC,NUM_SCALE,DSCRPTN
50,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[ZPLANT],[ZPLANT],[ZPLANT].[LEVEL01],,1,[20SALES_DIST],영업지역 (키),CHAR,006,006,,,영업지역 (키)
51,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[ZPLANT],[ZPLANT],[ZPLANT].[LEVEL01],,1,[10SALES_DIST],영업지역 (이름),CHAR,020,020,,,영업지역 (이름)
52,ZSDORM06,ZSDORM06/ZPB_SD_Q001,[ZPLANT],[ZPLANT],[ZPLANT].[LEVEL01],,1,[20SRCID],GeoLoct.데이타소스ID (키),CHAR,004,004,,,GeoLoct.데이타소스ID (키)


In [33]:
result = conn.call("RFC_READ_TABLE", 
                   QUERY_TABLE='ZSDORM06/ZPB_SD_Q001', 
#                    DIM_NAM = Measures, 
                   OPTIONS = [{'TEXT': "[ZDAY_1] = {20220901 - 20220902}"},
                              {'TEXT': "[!V000003] = '00000000000000000000000000010301'" }, 
                             ], 
                   DELIMITER = ",")

ABAPApplicationError: RFC_ABAP_EXCEPTION (rc=5): key=TABLE_NOT_AVAILABLE, message=ID:SV Type:E Number:029 ZPB_SD_Q001 [MSG: class=SV, type=E, number=029, v1-4:=ZPB_SD_Q001;;;]

In [ ]:
result = conn.call("RFC_READ_TABLE", QUERY_TABLE='MCHB', FIELDS = fields, OPTIONS = [{'TEXT': "WERKS = 'KR18'"}])

In [ ]:
= Cube.Transform(#"ZSDORM06/ZPB_SD_Q001",
        {
            {Cube.ApplyParameter, "[ZDAY_1]", {20220901 - 20220902}},
            {Cube.ApplyParameter, "[!V000003]", {"[ZMATERIAL__ZEIS_L1].[00000000000000000000000000010301]"}},
            {Cube.AddAndExpandDimensionColumn, "[ZMATERIAL__ZEIS_L1]", {"[ZMATERIAL__ZEIS_L1].[LEVEL01]"}, {"EIS 표준계층구조 1레벨.EIS 표준계층구조 1레벨 레벨 01"}},
            {Cube.AddAndExpandDimensionColumn, "[0CALDAY]", {"[0CALDAY].[LEVEL01]"}, {"달력일.달력일 레벨 01"}},
            {Cube.AddMeasureColumn, "수주금액", "[Measures].[9SMYY8SQSKECRELW0ZIZ7VDOW]"},
            {Cube.AddMeasureColumn, "수주수량", "[Measures].[9M8SI8WYOF6CBW27E0EEOWG7K]"}
        })

In [15]:
# fields =  ['ZSALEAMNT' ]
result = conn.call("BAPI_MATERIAL_GET_ALL")
print(result)

ABAPApplicationError: RFC_ABAP_EXCEPTION (rc=5): key=FU_NOT_FOUND, message=ID:FL Type:E Number:046 BAPI_MATERIAL_GET_ALL [MSG: class=FL, type=E, number=046, v1-4:=BAPI_MATERIAL_GET_ALL;;;]

In [25]:
fields =  ['ZSALEAMNT' ]
result = conn.call("RFC_READ_TABLE", QUERY_TABLE=u'ZSDOPLM01/ZPB_SD_Q014', FIELDS = fields
                   , OPTIONS = [{'TEXT': "ZC_PMO01 = '2022.10'"},
                                {'TEXT': "ZUSMLV01 = '00000000000000000000000000010301'" }, 
                                {'TEXT': "ZCALMON1 = '2022.09 - 2022.09'" } ], 
                   DELIMITER = ",")

{'TYPE': 'E', 'ID': 'RS_EXCEPTION', 'NUMBER': '000', 'MESSAGE': 'Exception CX_RSR_MDX_EMPTY occurred (program: CL_R SR_MDX_COMMAND============CP, include: CL_RSR_MDX_ COMMAND============CM001, line: 15).', 'LOG_NO': '', 'LOG_MSG_NO': '000000', 'MESSAGE_V1': 'Exception CX_RSR_MDX_EMPTY occurred (program: CL_R', 'MESSAGE_V2': 'SR_MDX_COMMAND============CP, include: CL_RSR_MDX_', 'MESSAGE_V3': 'COMMAND============CM001, line: 15).', 'MESSAGE_V4': '', 'PARAMETER': '', 'ROW': 0, 'FIELD': '', 'SYSTEM': 'BIPCLNT001'}


In [ ]:
# result = conn.call('BAPI_MDPROVIDER_GET_MEASURES')
# print (result)

In [ ]:
# result = conn.call('BAPI_MDPROVIDER_GET_DIMENSIONS')
# print (result)

In [79]:
# ???
result = conn.call('RSRTS_ACTIVATE_R3IS')
print (result)

ABAPApplicationError: RFC_ABAP_EXCEPTION (rc=5): key=FU_NOT_FOUND, message=ID:FL Type:E Number:046 RSRTS_ACTIVATE_R3IS [MSG: class=FL, type=E, number=046, v1-4:=RSRTS_ACTIVATE_R3IS;;;]

In [ ]:
= Cube.Transform(#"ZSDORM06/ZPB_SD_Q001",
        {
            {Cube.ApplyParameter, "[ZDAY_1]", {20220901 - 20220902}},
            {Cube.ApplyParameter, "[!V000003]", {"[ZMATERIAL__ZEIS_L1].[00000000000000000000000000010301]"}},
            {Cube.AddAndExpandDimensionColumn, "[ZMATERIAL__ZEIS_L1]", {"[ZMATERIAL__ZEIS_L1].[LEVEL01]"}, {"EIS 표준계층구조 1레벨.EIS 표준계층구조 1레벨 레벨 01"}},
            {Cube.AddAndExpandDimensionColumn, "[0CALDAY]", {"[0CALDAY].[LEVEL01]"}, {"달력일.달력일 레벨 01"}},
            {Cube.AddMeasureColumn, "수주금액", "[Measures].[9SMYY8SQSKECRELW0ZIZ7VDOW]"},
            {Cube.AddMeasureColumn, "수주수량", "[Measures].[9M8SI8WYOF6CBW27E0EEOWG7K]"}
        })